# Combine and clean spreadsheets

This notebook has Python code to parse, combine and clean the spreadsheets downloaded from [the FAA's website showing laser pointer incidents around the U.S.](https://www.faa.gov/about/initiatives/lasers/laws) The goal is to build a workflow that can be re-run any time the FAA releases new data and produce the same output: a single file, `faa-laser-incidents.csv`, that lands in the `class-notebooks` directory.

The process involves loading each spreadsheet into a pandas dataframe while conditionally applying some code to standardize its structure, then combining those separate dataframes into a single dataframe, then finally applying some code to clean up some of the data.


### One quick note
The process of deciding how each spreadsheet needs to be loaded and cleaned was iterative -- at first trying some sensible defaults to see what worked and what didn't, adjusting the process accordingly, testing changes on sample data before applying them to the entire dataset, and generally seeing how everything needed to fit together to build a common dataset. 

It took a couple of hours to get my head around the various problems with each spreadsheet and how things needed to fit together, but this two-steps-forward-one-step-back approach is pretty typical for sussing out data problems and then writing code to fix them.

It was only later, after I was more sure that I had a handle on things, that I thought more holistically about how the whole workflow should run and went back in and cleaned up various bits of test code, rearranged things, etc. (All of which to say: The process of developing a data workflow of any complexity is rarely linear.)

In [ ]:
# import from the Python standard library
import datetime

# import from a local python file
from fixes import fixes_injury, fixes_states, fixes_colors, problem_timestrings, problem_datestrings

# import from a package installed separately into a virtual environment
import pandas as pd

In [ ]:
'''
these are the common headers we'll be using for every year's dataframe -- I worked this out
by loading each spreadsheet into a dataframe and then looking at the .columns attribute
of each dataframe, then figured out which ones needed to be renamed, ignored, etc.
'''

headers_main = [
    'Incident Date',
    'Incident Time',
    'Flight ID',
    'Aircraft',
    'Altitude',
    'Airport',
    'Laser Color',
    'Injury',
    'City',
    'State'
]

### Load each year's spreadsheet individually

There's a little overlap, but each sheet needs something slightly different on import, so you need to express that in code somehow.

In [ ]:
# mapping key/value pairs of years to the files
xls_files = {
    '2010-2014': '../data/laser_incidents_2010-2014.xls',
    '2015': '../data/reported_laser_illumination_incidents_CY_2015.xls',
    '2016': '../data/reported_laser_illumination_incidents_CY_2016.xlsx',
    '2017': '../data/reported_laser_illumination_incidents_CY_2017.xlsx',
    '2018': '../data/Laser_Report_2018_final.xlsx',
    '2019': '../data/Laser_Report_2019_final.xlsx',
    '2020': '../data/Laser_Report_2020.xlsx',
    '2021': '../data/Laser-Report-2021-FINAL.xlsx',
    '2022': '../data/Laser-Report-2022-through-05-31.xlsx'   
}

In [ ]:
# 2010-2014 file needs a looooooot of work to standardize things

# sure, why not slightly rename your columns every single year
xls_1014_columns = {
    '2010': [
        'DATE',
        'TIME (UTC)',
        'ACID',
        'TYPE A/C',
        'ALT',
        'MAJOR CITY',
        'COLOR',
        'Injury Reported',
        'CITY',
        'STATE'
    ],
    '2011': [
        'DATE',
        'TIME (UTC)',
        'AC/ID',
        'TYPE A/C',
        'ALT',
        'MAJOR CITY',
        'COLOR',
        'Injury Reported',
        'CITY',
        'STATE'
    ],
    '2012': [
        'DATE',
        'TIME (UTC)',
        'Aircraft ID',
        'TYPE A/C',
        'ALT',
        'MAJOR CITY',
        'COLOR',
        'Injury Reported',
        'CITY',
        'STATE'
    ],
    '2013': [
        'DATE',
        'TIME (UTC)',
        'Aircraft ID',
        'TYPE A/C',
        'ALT',
        'MAJOR CITY',
        'COLOR',
        'Injury Reported',
        'CITY',
        'STATE'
    ],
    '2014': [
        'DATE',
        'TIME (UTC)',
        'Aircraft ID',
        'TYPE A/C',
        'ALT',
        'MAJOR CITY',
        'COLOR',
        'Injury Reported',
        'CITY',
        'STATE'
    ]
}

# we need to look at each sheet individually, so first we need to load it as an ExcelFile object
# https://pandas.pydata.org/docs/reference/api/pandas.ExcelFile.parse.html
xls_14 = pd.ExcelFile(xls_files['2010-2014'])

# create an empty data frame to start
df_14 = pd.DataFrame()

# loop over the years based on how the sheets are named: `Laser Report {year}`
# from 2010 - 2014
# (the range function is not inclusive on the back end)
# 
for year in range(2010, 2015):
    
    # create a new dataframe from this particular worksheet
    # and use the columns we mapped out above
    new_df = pd.read_excel(
        xls_14,
        f'Laser Report {year}',
        usecols=xls_1014_columns[str(year)]
    )
    
    # rename the columns to our standard values above
    new_df.rename(columns=dict(zip(xls_1014_columns[str(year)], headers_main)), inplace=True)
    
    # add it to the main df
    df_14 = pd.concat([df_14, new_df])

# kill out errant "Total" rows
df_14 = df_14[df_14['Flight ID'].str.contains('Total', na=False) == False]

In [ ]:
df_14.head()

In [ ]:
# lop off two crufty columns at the end
df_15 = pd.read_excel(
    xls_files['2015'],
    usecols=range(10),
    names=headers_main
)

In [ ]:
df_15.head()

In [ ]:
# standardize column names
df_16 = pd.read_excel(
    xls_files['2016'],
    usecols=headers_main
)

In [ ]:
df_16.head()

In [ ]:
# standardize column names
df_17 = pd.read_excel(
    xls_files['2017'],
    names=headers_main
)

In [ ]:
df_17.head()

In [ ]:
# standardize column names
df_18 = pd.read_excel(
    xls_files['2018'],
    names=headers_main
)

In [ ]:
df_18.head()

In [ ]:
df_19 = pd.read_excel(
    xls_files['2019'],
    names=headers_main
)

In [ ]:
df_19.head()

In [ ]:
# standardize column names
df_20 = pd.read_excel(
    xls_files['2020'],
    names=headers_main
)

In [ ]:
df_20.head()

In [ ]:
# skip crufty first row and standardize column names
df_21 = pd.read_excel(
    xls_files['2021'],
    skiprows=1,
    names=headers_main
)

In [ ]:
df_21.head()

In [ ]:
# standardize column names
df_22 = pd.read_excel(
    xls_files['2022'],
    names=headers_main
)

In [ ]:
df_22.head()

In [ ]:
# load all of these dataframes into a list for later convenience
dfs = [
    df_14,
    df_15,
    df_16,
    df_17,
    df_18,
    df_19,
    df_20,
    df_21,
    df_22
]

In [ ]:
# check that all column values are the same
[list(x.columns) == list(dfs[0].columns) for x in dfs]

In [ ]:
# a custom function to mash together the date and time columns into a proper datetime
def get_utc_datetime(row):
    
    # grab the date
    datestr = row['Incident Date']
    
    # check to see if that date is on our list of problems
    if datestr in problem_datestrings or not datestr:
        return datetime.datetime(0, 0, 0, 0, 0, 0)
    
    # make it an actual date, not a datetime
    date = datestr.date()

    # grab the time
    time = row['Incident Time']
    
    # check to see if this value is on our problem list
    if not time or time in problem_timestrings:
        time = '0000'
    else:
        try:
            
            # pad it out to four digits
            time = str(int(row['Incident Time'])).zfill(4).strip()
        except ValueError:
            time = '0000'
    
    # assemble a UTC date (confirmed w/ FAA press human)
    datetime_str = f'{date}T{time[:2]}:{time[2:]}:00Z'
    return datetime.datetime.strptime(datetime_str,'%Y-%m-%dT%H:%M:%SZ')

In [ ]:
# loop over the data frames and apply the function
for df in dfs:
    
    # use the function to generate a UTC datetime
    df['datetime_utc'] = df.apply(get_utc_datetime, axis=1)

In [ ]:
# smush all the data frames together into one
df = pd.concat(dfs)

In [ ]:
# extract the year into a new column
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.html
df['year'] = df['Incident Date'].dt.year

In [ ]:
# check your work
df.head()

In [ ]:
df.tail()

In [ ]:
# how many records altogether?
len(df)

In [ ]:
# check to see how dirty the Injury column is
list(df.Injury.unique())

In [ ]:
# strip whitespace and upcase
df.Injury = df.Injury.str.upper().str.strip()

In [ ]:
# now see unique values
sorted(set([x for x in list(df['Injury'].unique()) if pd.isnull(x) == False]))

In [ ]:
# what's the deal with this one?
df[df.Injury == 'GREEN']

In [ ]:
# apply a lambda function to look up the value (clean) by the key (messy)
df['injury_clean'] = df.apply(lambda row: fixes_injury.get(row['Injury'], row['Injury']), axis=1)

In [ ]:
df.injury_clean.value_counts()

In [ ]:
sorted(df.year.unique())

In [ ]:
# check flight ID
df['Flight ID'].value_counts()

In [ ]:
# standardize state values
df['State'] = df['State'].str.upper().str.strip()

In [ ]:
# check the list
sorted(set([x for x in list(df['State'].unique()) if pd.isnull(x) == False]))

In [ ]:
# apply a lambda function to look up the value (clean) by the key (messy)
df['state_clean'] = df.apply(lambda row: fixes_states.get(row['State'], row['State']), axis=1)

In [ ]:
df.head()

In [ ]:
df.state_clean.value_counts()

In [ ]:
# same routine with the laser color
df['Laser Color'] = df['Laser Color'].str.strip().str.lower()

In [ ]:
sorted(set([x for x in list(df['Laser Color'].unique()) if pd.isnull(x) == False]))

In [ ]:
# apply a lambda function to look up the value (clean) by the key (messy)
df['colors_clean'] = df.apply(lambda row: fixes_colors.get(row['Laser Color'], row['Laser Color']), axis=1)

In [ ]:
df.head()

In [ ]:
# export to CSV
df.to_csv('faa-laser-incidents.csv', index=False)